# __COSAN MLIP MD simulations__
Objective: generate enough sampling to determine if the experimental solvent-dependence of COSAN is observed using three different uMLIP models: UMA-small, MACE-MP-0b2 and ORB-v3. 

Secondary objective: See if in sampling the lack of rotational invariance of ORB models is diffuminated. Inspired by the ETH Zürich team regarding rotational invariance and direct force prediction. 

### __Model definitions__

In [3]:
# MACE
from mace.calculators import mace_mp

macemp = mace_mp(
    model="medium", 
    dispersion=False, 
    default_dtype="float32", 
    device='cpu',
    )

CALC_NAME = 'MACE-MP'

/Users/sergiortizropero/miniconda3/envs/ASE_all/lib/python3.10/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indice

cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.
Using Materials Project MACE for MACECalculator with /Users/sergiortizropero/.cache/mace/20231203mace128L1_epoch199model
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.
Default dtype float32 does not match model dtype float64, converting models to float32.


/Users/sergiortizropero/miniconda3/envs/ASE_all/lib/python3.10/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_p

In [1]:
# ORB
from orb_models.forcefield.pretrained import orb_v3_conservative_inf_omat
from orb_models.forcefield.calculator import ORBCalculator

orbff_v3 = orb_v3_conservative_inf_omat(device='cpu')
orbff_v3_calc = ORBCalculator(orbff_v3, device='cpu')

CALC_NAME = 'ORB'

/Users/sergiortizropero/miniconda3/envs/ASE_orbital/lib/python3.10/site-packages/orb_models/utils.py:30: UserWarning: Setting global torch default dtype to torch.float32.
  warnings.warn(f"Setting global torch default dtype to {torch_dtype}.")


In [1]:
# UMA
from fairchem.core import FAIRChemCalculator
from fairchem.core.units.mlip_unit import load_predict_unit

# define the UMA ASE calculator
uma_predictor = load_predict_unit(
    path='/Users/sergiortizropero/TFG_phys/NNPs_TFG/models/ase_uma/uma-s-1p1.pt', 
    device='cpu',                   
    inference_settings='default',   
)
uma_calc = FAIRChemCalculator(
    uma_predictor,
    task_name='omol',               
)

CALC_NAME = 'UMA'

/Users/sergiortizropero/miniconda3/envs/ASE_UMA_test/lib/python3.12/site-packages/torchtnt/utils/version.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
W0718 11:51:27.198000 84253 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


### __Imports & Functions__

In [41]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
rc('lines', lw=1, color='b')
rc('legend', loc='best')
plt.rcParams["legend.fancybox"] = False
plt.rcParams["legend.edgecolor"] = 'black'
plt.rcParams['legend.borderpad'] = 0.25
plt.rcParams['legend.fontsize'] = 11
plt.rcParams.update({'pgf.preamble': r'\usepackage{amsmath}'})

from ase.visualize import view
from ase.build import molecule
from ase.io import write, read
from ase.io import Trajectory, trajectory
from ase import build
from ase.data.pubchem import pubchem_atoms_search

import ase.units as units
from ase import Atoms
from ase.calculators.tip3p import TIP3P, angleHOH, rOH
from ase.constraints import FixBondLengths
from ase.io.trajectory import Trajectory
from ase.io.trajectory import TrajectoryReader
from ase.md import Langevin
from ase.md.nptberendsen import NPTBerendsen
from ase.optimize import BFGS
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.calculators.checkpoint import Checkpoint
import pickle

In [42]:
def prepare_cosan(atoms, save_path, sim_name, T_0, pbc=False, box_size=None):

    # add calculator
    if CALC_NAME == 'MACE-MP':
        atoms.calc = macemp
    elif CALC_NAME == 'ORB':
        atoms.calc = orbff_v3_calc
    elif CALC_NAME == 'UMA':
        atoms.calc = uma_calc
    else:
        raise ValueError('name of the calculation not identified!')

    # system preprocessing
    if box_size is not None:
        atoms.set_cell(box_size)
    else:
        atoms.set_cell((20, 20, 20))
    atoms.center()

    if pbc:
        atoms.set_pbc(True)

    view(atoms)

    # path setup
    sim_path = os.path.join(save_path, sim_name)

    if not os.path.exists(sim_path):
        os.makedirs(sim_path)

    # initialize velocities
    MaxwellBoltzmannDistribution(atoms, temperature_K=T_0)

    return atoms


    
def run_NVT(atoms, length, dt, T, sim_path, sim_name, k, save_int=100, overwrite=False):

    # define paths
    log_path = os.path.join(sim_path, sim_name + f'_NVT_{k}.log')
    traj_path = os.path.join(sim_path, sim_name + f'_NVT_{k}.traj')

    # NVT run
    md = Langevin(
        atoms,
        dt*units.fs,
        temperature_K=T,
        friction=0.01,
        logfile=log_path,
        loginterval=save_int,
    )

    # trajectory
    if os.path.exists(traj_path) and (not overwrite):
        raise ValueError('Trajectory file already exists!')
    traj = Trajectory(traj_path, 'w', atoms)

    # save every save_int steps
    md.attach(traj.write, interval=save_int)
    md.run(length)
    print(f'md {k} has finished')

    return atoms



def run_NPT(atoms, length, dt, T, P, k, sim_path, sim_name, save_int=100, overwrite=False):

    # define paths
    log_path = os.path.join(sim_path, sim_name + f'_NPT_{k}.log')
    traj_path = os.path.join(sim_path, sim_name + f'_NPT_{k}.traj')

    # NVT run
    md = NPTBerendsen(
        atoms,
        dt*units.fs,
        temperature_K=T,
        pressure=1,
        compressibility=4.6*10**-5,
        logfile=log_path,
        loginterval=save_int,
    )

    # trajectory
    if os.path.exists(traj_path) and (not overwrite):
        raise ValueError('Trajectory file already exists!')
    traj = Trajectory(traj_path, 'w', atoms)

    # save every save_int steps
    md.attach(traj.write, interval=save_int)
    md.run(length)
    print(f'md {k} has finished')

    return atoms


def run_prod(atoms, length, dt, T, k, sim_path, sim_name, save_int=100, overwrite=False):

    # define paths
    log_path = os.path.join(sim_path, sim_name + f'_PROD_{k}.log')
    traj_path = os.path.join(sim_path, sim_name + f'_PROD_{k}.traj')

    # NVT run
    md = Langevin(
        atoms,
        dt*units.fs,
        temperature_K=T,
        friction=0.01,
        logfile=log_path,
        loginterval=save_int,
    )

    # trajectory
    if os.path.exists(traj_path) and (not overwrite):
        raise ValueError('Trajectory file already exists!')
    traj = Trajectory(traj_path, 'w', atoms)

    # save every save_int steps
    md.attach(traj.write, interval=save_int)
    md.run(length)
    print(f'md {k} has finished')

    return atoms




def restart_md(pkl_path, verbose=True):
    
    with open(pkl_path, 'rb') as f:
        cosan_restart = pickle.load(f)

    if verbose:
        print('restarting with Atoms.object with properties:')
        print(cosan_restart.info['charge'])
        print(cosan_restart.info['spin'])
        print(cosan_restart.info['k'])
        #print(cosan_restart.info['positions'])
        #print(cosan_restart.info['velocities'])

    return cosan_restart




def relax_structure(atoms, sim_path, convergence=2):
    dyn = BFGS(atoms, trajectory=os.path.join(sim_path, 'relaxation.traj'))
    dyn.run(fmax=convergence)


# __COSAN MD__

In [ ]:
K = 10               # number of fragments for production
length = 10000      # in steps
dt = 1              # in fs
T_init = 300.       # in K
T_target = 300.     # in K
save_int = 100

ps_length = float(K * length * dt / 1000)
sim_name = f'{CALC_NAME}_{ps_length:.1f}'
save_path = '/Users/sergiortizropero/TFG_phys/COSAN_MLIP/simulations/cosan_mds'
sim_path = os.path.join(save_path, sim_name)

cosan_xyz = '/Users/sergiortizropero/TFG_phys/NNPs_TFG/scripts/symmetry/symmetries/cosan/cosan.xyz'
cosan = read(cosan_xyz)
cosan.info = {
    'spin': 3,
    'charge': -1,
    'k': 0,
}

restart = False
restart_file = ''

print(f'restarting: {restart}')
print(sim_name)
print('Performing {:.1f} ps NVT MD @ {} K'.format(ps_length, T_target))

restarting: True
MACE-MP_100.0
Performing 100.0 ps NVT MD @ 300.0 K


In [2]:
# define COSAN
prepare_cosan(
    cosan, 
    save_path, 
    sim_name, 
    T_init,
)

if restart:
    # retreive latest pickle.
    cosan = restart_md(restart_file, verbose=True)


converged = False
while not converged: 

    # start with k = 0
    # add 1
    cosan.info['k'] += 1
    k = cosan.info['k']

    # run a fragment
    # simulate k = 1
    run_NVT(
        cosan, 
        length, 
        dt, 
        T=T_target, 
        sim_path=sim_path, 
        sim_name=sim_name, 
        k=k, 
        save_int=save_int, 
        overwrite=restart,
    )

    # save to a file
    cosan.info['positions'] = cosan.get_positions()
    cosan.info['velocities'] = cosan.get_velocities()

    print(cosan.info['velocities'])
    print(cosan.info['k'])

    with open(os.path.join(sim_path, sim_name+f'_{k}_cosan.pkl'), 'wb') as f:
        pickle.dump(cosan, f)

    # check convergence
    if cosan.info['k'] >= K:
        converged = True

    

NameError: name 'cosan' is not defined

# __SOLVATED COSAN__

### Definition + Relaxation + NVT equilibration

In [46]:
K = 1               # number of fragments for production
length_nvt = 2     # in steps
length_npt = 2     # in steps
length_prod = 2    # in steps
dt = 1              # in fs

T_init = 300.       # in K
T_target = 300.     # in K
P_target = 1.       # in bar
save_int = 1


ps_length_nvt = float(K * length_nvt * dt / 1000)
ps_length_npt = float(K * length_npt * dt / 1000)
ps_length_prod = float(K * length_prod * dt / 1000)

sim_name = f'solvated_{CALC_NAME}_{ps_length_prod:.0f}'
save_path = '/Users/sergiortizropero/TFG_phys/NNPs_TFG/COSAN/simulations/cosan_mds'
sim_path = os.path.join(save_path, sim_name)

# initial solvated structure
cosan_pdb = '/Users/sergiortizropero/TFG_phys/NNPs_TFG/COSAN/simulations/cosan_solv/cosan_solv_ion.pdb'
cosan = read(cosan_pdb)
cosan.info = {
    'spin': 3,
    'charge': -1,
    'k_nvt': 0,
    'k_npt': 0,
    'k_prod': 0,
}


restart = False
restart_file = ''

print(f'restarting: {restart}')
print(sim_name)
print('Performing {:.1f} ps NVT MD @ {} K'.format(ps_length_prod, T_target))

restarting: False
solvated_MACE-MP_0
Performing 0.0 ps NVT MD @ 300.0 K


In [47]:
# DEFINITION AND RELAXATION
if not restart:
    # define COSAN  
    prepare_cosan(
        cosan, 
        save_path, 
        sim_name, 
        T_init,
        box_size=(28, 28, 32),
        pbc=True,
    )

    relax_structure(cosan, sim_path, convergence=5000)
    with open(os.path.join(sim_path, 'optimized.pkl'), 'wb') as f:
            pickle.dump(cosan, f)

else:
    # retreive latest pickle.
    cosan = restart_md(restart_file, verbose=True)

<class 'torch.Tensor'>
torch.float32
<class 'torch.Tensor'>
torch.float32
torch.float64
<class 'torch.Tensor'>
torch.float32
<class 'torch.Tensor'>
torch.float32
torch.float64
      Step     Time          Energy          fmax
BFGS:    0 20:29:39    -9854.343750      346.831563


In [48]:
# INITIAL NVT EQUILIBRATION

k = 0
converged_NVT = False
while not converged_NVT: 

    # start with k = 0
    # add 1
    cosan.info['k_nvt'] += 1
    k = cosan.info['k_nvt']

    # run a fragment
    # simulate k = 1
    run_NVT(
        cosan, 
        length_nvt, 
        dt, 
        T=T_target, 
        sim_path=sim_path, 
        sim_name=sim_name, 
        k=k, 
        save_int=save_int, 
        overwrite=restart,
    )

    # save to a file
    cosan.info['positions'] = cosan.get_positions()
    cosan.info['velocities'] = cosan.get_velocities()

    print(cosan.info['velocities'])
    print(cosan.info['k_nvt'])

    with open(os.path.join(sim_path, sim_name+f'_NVT_{k}_cosan.pkl'), 'wb') as f:
        pickle.dump(cosan, f)

    # check convergence
    if cosan.info['k_nvt'] >= K:
        converged_NVT = True    

<class 'torch.Tensor'>
torch.float32
<class 'torch.Tensor'>
torch.float32
torch.float64
<class 'torch.Tensor'>
torch.float32
<class 'torch.Tensor'>
torch.float32
torch.float64
<class 'torch.Tensor'>
torch.float32
<class 'torch.Tensor'>
torch.float32
torch.float64
<class 'torch.Tensor'>
torch.float32
<class 'torch.Tensor'>
torch.float32
torch.float64
md 1 has finished
[[ 5.54503799e-02 -1.16700216e-02 -5.21306930e-02]
 [ 2.85713209e-02 -1.01976578e-01  3.14327979e-02]
 [-1.35981908e-04 -4.27323613e-02 -1.71461732e-04]
 ...
 [ 5.62715916e-03 -2.89863336e-02  1.03336823e-02]
 [-2.99717750e-02 -2.60945316e-01  6.46518946e-03]
 [ 2.17749886e-01  1.64411418e-01 -6.47505984e-01]]
1


### NPT equilibration

In [49]:
# NPT EQUILIBRATION

k = 0
converged_NPT = False
while not converged_NPT: 

    # start with k = 0
    # add 1
    cosan.info['k_npt'] += 1
    k = cosan.info['k_npt']

    # run a fragment
    # simulate k = 1
    run_NPT(
        cosan, 
        length_npt, 
        dt, 
        T=T_target, 
        P=P_target,
        sim_path=sim_path, 
        sim_name=sim_name, 
        k=k, 
        save_int=save_int, 
        overwrite=restart,
    )

    # save to a file
    cosan.info['positions'] = cosan.get_positions()
    cosan.info['velocities'] = cosan.get_velocities()

    print(cosan.info['velocities'])
    print(cosan.info['k_npt'])

    with open(os.path.join(sim_path, sim_name+f'_NPT_{k}_cosan.pkl'), 'wb') as f:
        pickle.dump(cosan, f)

    # check convergence
    if cosan.info['k_npt'] >= K:
        converged_NPT = True    

<class 'torch.Tensor'>
torch.float32
<class 'torch.Tensor'>
torch.float32
torch.float64
<class 'torch.Tensor'>
torch.float32
<class 'torch.Tensor'>
torch.float32
torch.float64


/Users/sergiortizropero/miniconda3/envs/ASE_all/lib/python3.10/site-packages/ase/md/nptberendsen.py:223: FutureWarning: The 'pressure' parameter is deprecated, please specify the pressure in atomic units (eV/Å^3) using the 'pressure_au' parameter.
  warnings.warn(FutureWarning(w))
/Users/sergiortizropero/miniconda3/envs/ASE_all/lib/python3.10/site-packages/ase/md/nptberendsen.py:110: FutureWarning: Specify the compressibility in atomic units.
  warnings.warn(FutureWarning(


<class 'torch.Tensor'>
torch.float32
<class 'torch.Tensor'>
torch.float32
torch.float64
<class 'torch.Tensor'>
torch.float32
<class 'torch.Tensor'>
torch.float32
torch.float64
<class 'torch.Tensor'>
torch.float32
<class 'torch.Tensor'>
torch.float32
torch.float64
<class 'torch.Tensor'>
torch.float32
<class 'torch.Tensor'>
torch.float32
torch.float64
<class 'torch.Tensor'>
torch.float32
<class 'torch.Tensor'>
torch.float32
torch.float64
<class 'torch.Tensor'>
torch.float32
<class 'torch.Tensor'>
torch.float32
torch.float64
md 1 has finished
[[ 0.0481821  -0.00674403 -0.04812309]
 [ 0.02956118 -0.10183054  0.02640164]
 [ 0.00271217 -0.04278143 -0.00284226]
 ...
 [-0.0044793  -0.03570413 -0.01595001]
 [-0.05811946 -0.06875467  0.25372201]
 [ 0.40414147  0.11226717 -0.54307589]]
1


### NVT production

In [50]:
# NVT PRODUCTION

k = 0
converged_prod = False
while not converged_prod: 

    # start with k = 0
    # add 1
    cosan.info['k_prod'] += 1
    k = cosan.info['k_prod']

    # run a fragment
    # simulate k = 1
    run_prod(
        cosan, 
        length_prod, 
        dt, 
        T=T_target, 
        sim_path=sim_path, 
        sim_name=sim_name, 
        k=k, 
        save_int=save_int, 
        overwrite=restart,
    )

    # save to a file
    cosan.info['positions'] = cosan.get_positions()
    cosan.info['velocities'] = cosan.get_velocities()

    print(cosan.info['velocities'])
    print(cosan.info['k_prod'])

    with open(os.path.join(sim_path, sim_name+f'_PROD_{k}_cosan.pkl'), 'wb') as f:
        pickle.dump(cosan, f)

    # check convergence
    if cosan.info['k_prod'] >= K:
        converged_prod = True    

<class 'torch.Tensor'>
torch.float32
<class 'torch.Tensor'>
torch.float32
torch.float64
<class 'torch.Tensor'>
torch.float32
<class 'torch.Tensor'>
torch.float32
torch.float64
<class 'torch.Tensor'>
torch.float32
<class 'torch.Tensor'>
torch.float32
torch.float64
<class 'torch.Tensor'>
torch.float32
<class 'torch.Tensor'>
torch.float32
torch.float64
md 1 has finished
[[ 0.03787077 -0.00732138 -0.04636899]
 [ 0.02802821 -0.09568559  0.02459822]
 [ 0.00076819 -0.04870403 -0.00645133]
 ...
 [-0.01801926 -0.02753834 -0.04248052]
 [-0.06258994 -0.06043385  0.37078124]
 [ 0.57686727 -0.00352733 -0.31154778]]
1
